# Proyecto Final: Predicción de Riesgo de Crédito (Home Credit)

Este notebook consolida todo el flujo de trabajo del proyecto, desde la carga de datos hasta la evaluación del modelo, siguiendo la metodología CRISP-DM.

## 1. Configuración e Importaciones
Primero, importamos las librerías necesarias y configuramos el entorno.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import gc
import os
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, confusion_matrix, roc_curve, classification_report

# Configuración de visualización
plt.style.use('ggplot')
pd.set_option('display.max_columns', 100)

## 2. Preparación de Datos (Feature Engineering)

En esta etapa, cargamos los datos crudos, realizamos limpieza, one-hot encoding y agregaciones. 
Por eficiencia, el código pesado se encuentra modularizado en `02_data_preparation/feature_engineering.py`. 
Aquí cargaremos directamente el dataset procesado `processed_data.parquet` si ya existe, o ejecutaremos una versión simplificada.

In [ ]:
# Ruta al dataset procesado
PROCESSED_DATA_PATH = '../processed_data.parquet'

if os.path.exists(PROCESSED_DATA_PATH):
    print("Cargando dataset procesado...")
    df = pd.read_parquet(PROCESSED_DATA_PATH)
    print(f"Dataset cargado: {df.shape}")
else:
    print("El dataset procesado no existe. Por favor ejecute el script de feature engineering o asegúrese de que el archivo existe.")
    # Opcional: Llamar al script aquí si se desea
    # !python 02_data_preparation/feature_engineering.py

## 3. Modelado (LightGBM)

Entrenamos un modelo LightGBM (Gradient Boosting Machine) optimizado para velocidad y rendimiento. Utilizamos `is_unbalance=True` para manejar la clase minoritaria (impagos).

In [ ]:
# Filtrar datos de entrenamiento (donde TARGET no es nulo)
train_df = df[df['TARGET'].notnull()]

y = train_df['TARGET']
X = train_df.drop(columns=['TARGET', 'SK_ID_CURR'])

# Limpieza de nombres de columnas para LightGBM (sin caracteres especiales)
X.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in X.columns]

print(f"Dimensiones de entrenamiento: {X.shape}")

# Split Train/Validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Dataset de LightGBM
dtrain = lgb.Dataset(X_train, label=y_train)
dval = lgb.Dataset(X_val, label=y_val, reference=dtrain)

# Hiperparámetros
params = {
    'objective': 'binary',
    'metric': 'auc',
    'is_unbalance': True,
    'boosting_type': 'gbdt',
    'learning_rate': 0.02,
    'num_leaves': 31,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1
}

print("Entrenando modelo...")
model = lgb.train(
    params,
    dtrain,
    num_boost_round=1000,
    valid_sets=[dtrain, dval],
    callbacks=[lgb.early_stopping(stopping_rounds=100), lgb.log_evaluation(100)]
)

## 4. Evaluación del Modelo

Analizamos el rendimiento del modelo utilizando AUC, Matriz de Confusión y Curva ROC.

In [ ]:
# Predicciones
y_pred_prob = model.predict(X_val, num_iteration=model.best_iteration)
y_pred_class = (y_pred_prob > 0.5).astype(int)

# AUC Score
auc = roc_auc_score(y_val, y_pred_prob)
print(f"AUC de Validación: {auc:.4f}")

# Matriz de Confusión
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_val, y_pred_class)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Matriz de Confusión')
plt.ylabel('Etiqueta Real')
plt.xlabel('Etiqueta Predicha')
plt.show()

# Curva ROC
fpr, tpr, _ = roc_curve(y_val, y_pred_prob)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'AUC = {auc:.4f}')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('Tasa de Falsos Positivos')
plt.ylabel('Tasa de Verdaderos Positivos')
plt.title('Curva ROC')
plt.legend()
plt.show()

### Importancia de Variables
¿Qué características influyen más en la decisión del modelo?

In [ ]:
lgb.plot_importance(model, max_num_features=20, importance_type='split', figsize=(10, 8))
plt.title('Top 20 Variables Más Importantes')
plt.show()

## 5. Simulación de API (Inferencia)

Simulamos cómo funcionaría el endpoint `/evaluate_risk` con un cliente nuevo.

In [ ]:
def evaluate_client(client_data, model, feature_names):
    # Crear DataFrame y alinear columnas
    input_df = pd.DataFrame([client_data])
    
    # Asegurar que todas las columnas del modelo existan (rellenar con 0 o NaN)
    input_df = input_df.reindex(columns=feature_names)
    
    # Predecir
    prob = model.predict(input_df)[0]
    
    decision = "REVISIÓN MANUAL"
    if prob < 0.08: decision = "APROBAR"
    elif prob > 0.3: decision = "RECHAZAR"
        
    return prob, decision

# Tomar un ejemplo del set de validación
sample_client = X_val.iloc[0].to_dict()
prob, decision = evaluate_client(sample_client, model, X.columns.tolist())

print(f"Cliente Ejemplo:")
print(f"Probabilidad de Impago: {prob:.2%}")
print(f"Decisión Sugerida: {decision}")